In [224]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

train = pd.read_excel('data_column_train_matlab.xlsx')
train_data = train.to_numpy()
label = pd.read_excel('data_label_trian_matlab.xlsx')
label_data = label.to_numpy()
final_data = np.concatenate((train_data,label_data),axis =1)

In [225]:
shuffled_data = np.random.shuffle(final_data)
df = pd.DataFrame(data=final_data)
X_train = final_data[:,0:32]
Y_train = final_data[:,32:]
m = X_train.shape[0]
layer_dims = [32,10,5]
X_train = (X_train-X_train.mean())/X_train.std()


test = pd.read_excel('Test sample set.xlsx')
test1 = test.drop(['fault types'],axis=1)

test2 = test1.drop(['test results'],axis=1)
test3 = test2.to_numpy()
X_test = test3[:,1:]
Y_test = test3[:,0:1]
X_test = (X_test-X_test.mean())/(X_test.std())

m = Y_test.shape[0]
Y_one_hot = np.zeros((m,5))
for i in range(m):
    j = int(Y_test[i])
    Y_one_hot[i,j-1] = Y_one_hot[i,j-1] + 1

In [226]:
def initialize_parameters(layer_dims):
    #layer1=32
    #layer2=10
    #layer3=5
    parameters={}
    L = len(layer_dims)
    np.random.seed(1)
    
    for l in range(1,L):
        parameters['W'+str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b'+str(l)] = np.zeros((layer_dims[l],1))
    return parameters

In [227]:
# helper function
def relu(x):
    '''
    return np.maximum(0,matrix)
    '''
    # np.where(matrix > 0, matrix, matrix * 0.01)
    y1 = ((x > 0) * x)                                                 
    y2 = ((x <= 0) * x * 0.01)                                         
    leaky_way2 = y1 + y2
    return leaky_way2
    
def sigmoid(matrix):
    return 1/(1+np.exp(-matrix))

def softmax(matrix):
    e_x = np.exp(matrix - np.max(matrix))
    return e_x/e_x.sum(axis=0)

def relu_derivative(x):
    
    x[x<0] = 0.01
    x[x>=0] = 1
    
    return x

def sigmoid_derivative(x):
    df = sigmoid(x)
    return df*(1-df)

In [324]:
def forward_propogation(parameters,X_train):
    
    cache = {}
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    Z1 = W1@X_train.T + b1
    A1 = relu(Z1)
    Z2 = W2@A1 + b2
    A2 = sigmoid(Z2)
    
    cache = {'Z1':Z1,'A1':A1,'Z2':Z2,'A2':A2}
    return cache


In [325]:
def compute_cost(cache,parameters,X_train,Y_train):
    W1 = parameters['W1']
    W2 = parameters['W2']
    A2 = cache['A2']
    m = X_train.shape
    lambd=0 
    cross_entropy = -np.mean(Y_train*np.log(A2.T+np.exp(-8)))
    #L2_cost = (np.sum(np.square(W1))+np.sum(np.square(W2)))*(np.divide(lambd,2*m))
    cost = cross_entropy # L2_cost
    
    
    return cost


In [326]:
def backward_propogation(cache,parameters,Y_train,X_train):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    lambd =0
    
    Z1 = cache['Z1']
    A1 = cache['A1']
    Z2 = cache['Z2']
    A2 = cache['A2']
    m = X_train.shape[0]
    gradients = {}
    
    dZ2 = A2 - Y_train.T
    dW2 = (1/m)*dZ2@A1.T  #(lambd/m)*W2
    db2 = (1/m)*np.sum(dZ2,axis=1,keepdims=True)
    dZ1 = W2.T@dZ2*relu_derivative(Z1)
    dW1 = (1/m)*dZ1@X_train # (lambd/m)*W1
    db1 = (1/m)*np.sum(dZ1,axis=1,keepdims=True)
    
    gradients = {'dZ2':dZ2,'dW2':W2,'db2':db2,'dZ1':dZ1,'dW1':dW1,'db1':db1}
    
    return gradients




In [327]:
def optimize(layer_dims,parameters,gradients,learning_rate):
    L = len(layer_dims)
    Vdw = 0
    Vdb = 0
    Sdw = 0
    Sdb = 0
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 0.00000001
    
    for l in range(1,L):
        '''
        Vdw = beta1*Vdw +(1-beta1)*parameters['W'+str(l)]
        Vdb = beta1*Vdb +(1-beta1)*parameters['b'+str(l)]
        Sdw = beta1*Sdw +(1-beta2)*(parameters['W'+str(l)]**2)
        Sdb = beta1*Sdb +(1-beta2)*(parameters['b'+str(l)]**2)
        
        Vdw_corrected = Vdw/(1-beta1**l)
        Vdb_corrected = Vdb/(1-beta1**l)
        Sdw_corrected = Sdw/(1-beta2**l)
        Vdb_corrected = Vdb/(1-beta2**l)
        
        
        parameters['W'+str(l)] = parameters['W'+str(l)] - learning_rate*Vdw_corrected/(math.sqrt(Sdw_corrected)+epsilon)
        parameters['b'+str(l)] = parameters['b'+str(l)] - learning_rate*Vdb_corrected/(math.sqrt(Sdb_corrected)+epsilon)
        '''
        parameters['W'+str(l)] = parameters['W'+str(l)] - learning_rate*parameters['W'+str(l)]
        parameters['b'+str(l)] = parameters['b'+str(l)] - learning_rate*parameters['b'+str(l)]
        
    return parameters



In [328]:
def model(X,Y,layer_dims,learning_rate,num_iterations):
    np.random.seed(1)
    grads = {}
    costs = []
    m = X.shape[0]
    
    parameters = initialize_parameters(layer_dims)
    
    for i in range(0,num_iterations):
        
        cache = forward_propogation(parameters,X)
        cost = compute_cost(cache,parameters,X,Y)
        gradients = backward_propogation(cache,parameters,Y,X)
        
        parameters = optimize(layer_dims,parameters,gradients,learning_rate)
        costs.append(cost)
        if i%10 == 0:
            print('Cost after iteration {}: {}'.format(i,cost))
            
    return parameters,costs

In [339]:
final_params,costs = model(X_train,Y_train,layer_dims=[32,10,5],learning_rate=0.0075,num_iterations=2000)

In [335]:
def predict(params, X,Y):
    accuracy = []
    cache = forward_propogation(params,X)
    Z1 = cache['Z1']
    A1 = cache['A1']
    Z2 = cache['Z2']
    A2 = cache['A2']
    
    diff = Y.T - A2
    for i in range(diff.shape[1]):
        loss = np.max(diff[:,i])*100
        acc = 100 - loss
        accuracy.append(acc)
    
    return accuracy,A2

In [331]:
bol,A = predict(final_params,X_test,Y_one_hot)